In [2]:
import numpy as np
import keras
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [5]:
data = np.loadtxt("data.txt")

In [6]:
data.shape

(7556, 181)

In [7]:
# generate training set and test set

dataX = data[:,0:-1]
dataY = data[:,-1]

In [8]:
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size = 0.33, shuffle = True)

In [18]:
dropout_rate = 0

input_dim = trainX.shape[0]
model = Sequential()
model.add(Dense(32, activation = "elu", input_dim = 180))
model.add(Dropout(dropout_rate))
model.add(Dense(16, activation = "elu"))
model.add(Dropout(dropout_rate))
model.add(Dense(8, activation = "elu"))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation = "elu"))
model.compile(loss = "mean_absolute_error", optimizer = "adam") 
# mean_absolute_error, mean_squared_error;
# SGD, adam, RMSprop, Adagrad, Adadelta, Adamax, Nadam
train_history = model.fit(x = trainX, y = trainY, 
                                    epochs = 100, batch_size = 16, 
                                    validation_data = (testX, testY))

Train on 5062 samples, validate on 2494 samples
Epoch 1/100
5062/5062 [==============================] - 2s 493us/step - loss: 247.7998 - val_loss: 267.9251
Epoch 2/100
5062/5062 [==============================] - 1s 179us/step - loss: 237.3580 - val_loss: 261.5666
Epoch 3/100
5062/5062 [==============================] - 1s 182us/step - loss: 232.2062 - val_loss: 262.2710
Epoch 4/100
5062/5062 [==============================] - 1s 180us/step - loss: 232.3206 - val_loss: 260.3610
Epoch 5/100
5062/5062 [==============================] - 1s 188us/step - loss: 234.3109 - val_loss: 258.6633
Epoch 6/100
5062/5062 [==============================] - 1s 180us/step - loss: 231.1973 - val_loss: 256.1472
Epoch 7/100
5062/5062 [==============================] - 1s 185us/step - loss: 230.8557 - val_loss: 260.5133
Epoch 8/100
5062/5062 [==============================] - 1s 181us/step - loss: 228.9567 - val_loss: 259.5805
Epoch 9/100
5062/5062 [==============================] - 1s 194us/step - loss: 2

5062/5062 [==============================] - 1s 193us/step - loss: 218.4600 - val_loss: 248.3836
Epoch 76/100
5062/5062 [==============================] - 1s 174us/step - loss: 218.6617 - val_loss: 251.1107
Epoch 77/100
5062/5062 [==============================] - 1s 170us/step - loss: 218.0826 - val_loss: 253.1108
Epoch 78/100
5062/5062 [==============================] - 1s 167us/step - loss: 217.9989 - val_loss: 248.7579
Epoch 79/100
5062/5062 [==============================] - 1s 166us/step - loss: 217.7681 - val_loss: 254.0711
Epoch 80/100
5062/5062 [==============================] - 1s 174us/step - loss: 218.4652 - val_loss: 251.4260
Epoch 81/100
5062/5062 [==============================] - 1s 172us/step - loss: 218.5019 - val_loss: 247.2835
Epoch 82/100
5062/5062 [==============================] - 1s 183us/step - loss: 218.5148 - val_loss: 249.2132
Epoch 83/100
5062/5062 [==============================] - 1s 171us/step - loss: 217.3253 - val_loss: 249.1163
Epoch 84/100
5062/5062 

In [15]:
min(trainY)

0.04991243502339581

In [3]:
predict_train = model.predict(trainX)

NameError: name 'model' is not defined

In [ ]:
predict_trainX.sha

In [112]:
trainY

array([613.42371783, 108.18828742,  40.13737483, ...,  58.00846539,
        52.16037587, 585.49247664])

In [1]:
plt.plot(trainX, predict_trainX )

NameError: name 'plt' is not defined

In [103]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                81008     
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_4 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 81,153
Trainable params: 81,153
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def NN():
    trainX = None
    trainY = None
    testX = None
    testY = None
    model = None
    score = None
    
    def __init__ (self,
                  dropout_layer_rate = 0.1,
                  rnn_dropout_rate = 0.1,
                  nb_epoch = 50,
                  batch_size = 16,
                  loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  save_model = True,
                  save_model_path = ''):

        self.dropout_layer_rate = dropout_layer_rate
        self.rnn_dropout_rate = rnn_dropout_rate
        self.nb_epoch = nb_epoch
        self.batch_size = batch_size
        self.loss = loss
        self.optimizer = optimizer #rmsprop, adam
        self.save_model = save_model
        self.save_model_path = save_model_path
        
    def NN_getData(self, trainX, trainY, testX, testY):
        self.trainX = trainX
        self.trainY = trainY
        self.testX = testX
        self.testY = testY

        return True
    
    def NN_model_train(self):
        trainX = self.trainX
        trainY = self.trainY
        testX = self.testX
        testY = self.testY
        model = self.model
        dropout_layer_rate = self.dropout_layer_rate
        rnn_dropout_rate = self.rnn_dropout_rate
        
        input_dim = trainX.shape
        
        model = Sequential()
        model.add()